In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import tensorflow as tf
import numpy as np
import scipy.misc
import scipy
import random
import os
from tensorflow.examples.tutorials.mnist import input_data

os.system("taskset -p 0xff %d" % os.getpid())

sess = tf.InteractiveSession()

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

conv1_act = np.load('Conv1_Activations_5000_good.npy')[:1,:,:,:4]

C = 1
M = 2
N = 4

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def residuals(x, mnist, conv1_act, N, M):
    a = x[:N*M].reshape(N,M)
    s = x[N*M: N*M + 5*5*M]
    s = s.reshape(5,5,M)
    b = x[5*5*M + N*M:]
    w_shape = (s.shape[0], s.shape[1], a.shape[0])
    #print(w_shape)
    w = np.zeros(w_shape)
    for i in range(a.shape[0]):
        aw = a[i][:]
        temp_w = np.zeros(s[:,:,0].shape)
        for j in range(a.shape[1]):
            temp_w += aw[j]*s[:,:,j]
        w[:,:,i] = temp_w
        
    x = tf.placeholder(tf.float32, shape=[None, 784])
    x_image = tf.reshape(x, [-1,28,28,1])
    b = b.astype(dtype=np.float32)
    b_tf = tf.constant(b)
    w = w.astype(dtype=np.float32)
    ws = w.shape
    w = w.reshape(ws[0], ws[1], 1, ws[2])
    weights_tf = tf.constant(w)
    conv1_pred = tf.nn.relu(conv2d(x_image, weights_tf) + b_tf)
    conv1_pred_np = conv1_pred.eval(feed_dict={x: mnist.train.images[:1]})
    
    diff = conv1_act - conv1_pred_np
    diff = np.multiply(diff, diff)
    norm_cost = 0.0
    for i in range(s.shape[2]):
        norm_cost += np.linalg.norm(s[:,:,i], ord='nuc')
    cost = 0.001*np.sum(diff) + 0.00001*np.sum(np.multiply(w,w)) + norm_cost
    return cost


a0 = np.random.randn(N,M)
w0 = np.random.randn(5,5,M)
b0 = np.random.randn(N,)

x0 = np.concatenate((a0.reshape(N*M), w0.reshape(5*5*M),b0))


xf = scipy.optimize.minimize(residuals, x0=x0, method='CG', args=(mnist, conv1_act, N, M)
                             ,options={'disp':True, 'maxiter':20})


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


KeyboardInterrupt: 

In [ ]:
xf.x[2]

In [ ]:
residuals(xf.x, mnist, conv1_act)

In [ ]:
xf.message

In [ ]:
residuals(x0, mnist, conv1_act)